In [ ]:
import wallaroo
import os

# Connect to Wallaroo client

In [ ]:
wl = wallaroo.Client()

In [ ]:
for pipeline in wl.list_pipelines():
    pipeline.undeploy()

# Choose the workspace to work in

In [ ]:
wl.list_workspaces()

In [ ]:
wl.set_current_workspace(wl.list_workspaces()[0])

# Upload models and deploy the Pipelines

In [ ]:
ccfraud_pp = wl.upload_model('ccfraud-demo' , "./ccfraud.onnx").deploy('ccfraud-demo')

In [ ]:
wl.list_pipelines()

In [ ]:
ccfraud_pp.undeploy()

# Inference Endpoint URL

In [ ]:
ccfraud_pp._deployment._url()

# Verify

In [ ]:
ccfraud_pp.infer_from_file('./high_fraud.json')

# Let's replace the model

In [ ]:
model2 = wl.upload_model('ccfraud-xgb-ccfrauddemo', './xgboost_ccfraud.onnx')
ccfraud_pp.replace_with_model_step(0, model2).deploy()

In [ ]:
logs = ccfraud_pp.logs(limit=10)
logs